In [1]:
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from perceptronac.convex_hull import convex_hull

In [7]:
def points_in_convex_hull(fname):

    data = pd.read_csv(fname,index_col="topology")
    normalized_data = data.copy()
    normalized_data["complexity"] = normalized_data["complexity"].apply(lambda x : math.log10(x)) # complexity in logarithmic scale
    normalized_data = MinMaxScaler().fit_transform(normalized_data.values) # make the coordinates in the range [0,1]

    plt.plot(normalized_data[:,0],normalized_data[:,1],linestyle="",marker="x") # verify that everything is alright

    chull = normalized_data[convex_hull(normalized_data.tolist()),:]
    plt.plot(normalized_data[:,0],normalized_data[:,1],linestyle="",marker="x")
    plt.plot(chull[:,0],chull[:,1],linestyle="solid",color="red",marker=None)

    plt.show()

    return data.iloc[convex_hull(normalized_data.tolist()),:]

In [1]:

points_in_convex_hull("/home/lucas/Documents/perceptronac/results/exp_1656174158/exp_1656174158_train_values.csv")


In [2]:
points_in_convex_hull("/home/lucas/Documents/perceptronac/results/exp_1656174158/exp_1656174158_valid_values.csv")

In [1]:
import torch
from perceptronac.models import ArbitraryMLP
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [68]:
def get_model_parameters_values(model):
    all_parameters = []
    for i in range(len(model.layers)):
        if isinstance(model.layers[i], torch.nn.Linear):
            with torch.no_grad():
                all_parameters.append(model.layers[i].weight.data.detach().numpy().reshape(-1))
                all_parameters.append(model.layers[i].bias.data.detach().numpy().reshape(-1))
    all_parameters = np.concatenate(all_parameters,axis=0)
    return all_parameters

def parameters_histogram(model):

    all_parameters = get_model_parameters_values(model)

    max_abs_value = np.max(np.abs(all_parameters))
    
    y,x = np.histogram(all_parameters,np.linspace(-max_abs_value,max_abs_value,100))
    plt.plot(x[:-1],y)

def estimate_midtread_uniform_quantization_delta(model,n_bits):
    """
    https://stackoverflow.com/questions/63582590/why-do-we-call-detach-before-calling-numpy-on-a-pytorch-tensor
    https://stackoverflow.com/questions/56816241/difference-between-detach-and-with-torch-nograd-in-pytorch    
    """

    all_parameters = get_model_parameters_values(model)

    max_abs_value = np.max(np.abs(all_parameters))

    # Delta = (max_abs_value - (-max_abs_value))/(2**n_bits) # midrise

    Delta = max_abs_value /(2**(n_bits-1)+(1/2)) # midtread

    return Delta

def midtread_uniform_quantization(model,Delta):
    for i in range(len(model.layers)):
        if isinstance(model.layers[i], torch.nn.Linear):
            with torch.no_grad():
                model.layers[i].weight.data = Delta * torch.round(model.layers[i].weight.data/Delta)
                model.layers[i].bias.data = Delta * torch.round(model.layers[i].bias.data/Delta)
    return model

def visualize_parameters_quantization_process(all_parameters,Delta):

    print("Delta")
    print(Delta)

    print("original")
    print(all_parameters)
    
    print("quantized")
    print(np.round(all_parameters/Delta))

    print("dequantized")
    print(Delta * np.round(all_parameters/Delta))

def test_midtread_uniform_quantization(model,n_bits=8):

    parameters_histogram(model)

    Delta = estimate_midtread_uniform_quantization_delta(model,n_bits)

    all_parameters = get_model_parameters_values(model)

    visualize_parameters_quantization_process(all_parameters,Delta)

    model2 = midtread_uniform_quantization(model,Delta)

    all_parameters2 = get_model_parameters_values(model2)

    assert np.allclose( all_parameters2, Delta * np.round(all_parameters/Delta) )

In [69]:

from perceptronac.perfect_AC import perfect_AC_binary
from perceptronac.models import StaticAC

def int_to_bits(v):
    """
    Sign-Magnitude representation
    
    https://www3.ntu.edu.sg/home/ehchua/programming/java/datarepresentation.html
    https://en.wikipedia.org/wiki/Two%27s_complement
    https://stackoverflow.com/questions/699866/python-int-to-binary-string

    """
    sign = 1 if v < 0 else 0
    bitseq = [int(c) for c in "{:08b}".format(np.abs(v))]
    bitseq[0] = sign
    return bitseq

def entropy(seq):
    p = np.zeros(len(np.unique(seq)))
    for i,v in enumerate(np.unique(seq)):
        p[i] = np.sum(seq == v)/len(seq)
    # p = np.histogram(quantized.flatten(), bins=np.arange(quantized.min(), quantized.max()+1))[0]
    # p = np.delete(p, p==0)
    # p = p / len(seq)
    return -(p * np.log2(p)).sum()


def encode_network_binary_symbols(values):

    bitstream = np.array([int_to_bits(v) for v in values]).reshape(-1)

    staticac = StaticAC()

    staticac.load(bitstream.reshape(-1,1))

    rate = perfect_AC_binary(bitstream.reshape(-1,1),staticac(bitstream.reshape(-1,1)))

    return rate


def encode_network_integer_symbols(quantized):

    rate = entropy(quantized)

    return rate


In [70]:
configs = pd.read_csv("/home/lucas/Documents/perceptronac/results/exp_1656174158/exp_1656174158_conf.csv",index_col="key").to_dict()["value"]
configs['save_dir'] = "/home/lucas/Documents/perceptronac/results/"
topologies = ast.literal_eval(configs["topologies"])

In [71]:
widths = topologies[0]
file_name = f"{configs['save_dir'].rstrip('/')}/exp_{configs['id']}/exp_{configs['id']}_{'_'.join(map(str,widths))}_min_valid_loss_model.pt"

In [72]:
model = ArbitraryMLP(widths)
model.load_state_dict(torch.load(file_name))

<All keys matched successfully>

In [79]:
# test_midtread_uniform_quantization(model)

In [74]:
Delta = estimate_midtread_uniform_quantization_delta(model,8)

In [75]:
model2 = midtread_uniform_quantization(model,Delta)

In [76]:
encode_network_binary_symbols(np.round(get_model_parameters_values(model)/Delta).astype(int))

0.965935401710126

In [77]:
encode_network_integer_symbols(np.round(get_model_parameters_values(model)/Delta).astype(int))

6.666744526778668